In [1]:
from __future__ import print_function, division

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict


In [2]:
EPISODES = 10000
Gamma = 0.9
Epsilon = 0.3
POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')


In [3]:
# writing the print functions
def print_Q(Q, grid):
    print('----------------------------------------------')
    print('| State  | U      | D      | L      | R      |')
    print('----------------------------------------------')
    all_states = sorted(grid.all_states())
    for state in all_states:
        print("| {} |".format(state), end="")
        if state != (0, 0) and state != (3, 3):
            for action, value in Q[state].items():
                print(' {:>6.2f} |'.format(value), end="")
        else:
            print('   0.00 |   0.00 |   0.00 |   0.00 |', end="")
        print('')
    print('----------------------------------------------')


def print_values(V, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")  # -ve sign takes up an extra space
        print("")

In [4]:

# in this code we also try to minimize the number of stepa involved to reach the terminal


In [5]:
# ======================== The Grid Class Starts Here ============================


class Grid:  # Environment
    def __init__(self, rows, cols, start):
        self.rows = rows
        self.cols = cols
        # since start (2,0) so, we are takig it as an array. At index 0 we have 2 and at index 1 we have 0.
        self.i = start[0]
        self.j = start[1]

    def set(self, rewards, actions):
        # rewards should be a dict of: (i, j): r (row, col): reward
        # actions should be a dict of: (i, j): A (row, col): list of possible actions
        self.rewards = rewards
        self.actions = actions

    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]

    def current_state(self):
        return (self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions

    def move(self, action):
        # check if legal move first
        #print('before i={} j={} action={}'.format(self.i, self.j, action))
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
        # return a reward (if any)
        reward = self.rewards.get((self.i, self.j), 0)
        #print('after i={} j={} r={}'.format(self.i, self.j, reward))
        return reward

    def undo_move(self, action):
        # these are the opposite of what U/D/L/R should normally do
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
        # raise an exception if we arrive somewhere we shouldn't be
        # should never happen
        assert(self.current_state() in self.all_states())

    def game_over(self):
        # returns true if game is over, else false
        # true if we are in a state where no actions are possible
        return (self.i, self.j) not in self.actions

    def all_states(self):
        # possibly buggy but simple way to get all states
        # either a position that has possible next actions
        # or a position that yields a reward
        return set(self.actions.keys()) | set(self.rewards.keys())

# ======================== The Grid Class Ends Here ============================

In [6]:
def max_dict(d):
    # choosing the maximum value of the state
    max_key = None  # key is whether it is U/D/L/R
    max_val = float('-inf')
    for k, v in d.items():
        if v > max_val:
            max_val = v
            max_key = k
    return max_key, max_val


In [7]:

def policy_using_pi(St, pi):
    return np.random.choice(POSSIBLE_ACTIONS, p=[pi[(St, a)] for a in POSSIBLE_ACTIONS])

In [8]:
def play_episode(grid, policy, pi):

    s = (2, 0)  # we start from (2, 0)
    grid.set_state(s)  # from self.i = 2 and self.j = 0
    a = policy_using_pi(s, pi)  # has a random choice of U/D/R/L

    # state reward -> we take an action from one state and land on to another state and recieve
    # a reward -> so from state s -> action a -> reward r
    # but initially state s by taking action a the reward is set to zero (0)
    states_actions_rewards = [(s, a, 0)]
    while True:
        r = grid.move(a)
        s = grid.current_state()
        if grid.game_over():
            # once game over no action to be taken
            states_actions_rewards.append((s, None, r))
            break
        else:
            a = policy_using_pi(s, pi)
            states_actions_rewards.append((s, a, r))

    # calculate the returns by working backwards from the terminal state
    G = 0
    states_actions_returns = []
    first = True
    for s, a, r in reversed(states_actions_rewards):
        # the value of the terminal state is 0 by definition
        # we should ignore the first state we encounter
        # and ignore the last G, which is meaningless since it doesn't correspond to any move
        if first:
            first = False
        else:
            states_actions_returns.append((s, a, G))
        G = r + Gamma * G
    states_actions_returns.reverse()  # we want it to be in order of state visited
    return states_actions_returns

In [9]:
def standard_grid():
        # define a grid that describes the reward for arriving at each state
        # and possible actions at each state
        # the grid looks like this

        # S means start position
        # E means the end states

        # E  .  .  .
        # .  .  . .
        # S  .  .  .
        # .  .  .  E

        # here row = 4 and col = 4 and start position is at (2,0)
    g = Grid(4, 4, (2, 0))
    # end points are at (0,0) and (3,3) so the reward at that point is zero ==> 0
    rewards = {(0, 0): 0, (3, 3): 0}
    actions = {
        # initially these actions are randomly defined. Like exploration. The agent taking random action at first.
        #(0, 0) ==> End-State
        (0, 1): ('D', 'R', 'L'),
        (0, 2): ('D', 'R', 'L'),
        (0, 3): ('D', 'L'),
        (1, 0): ('D', 'R', 'U'),
        (1, 1): ('D', 'R', 'L', 'U'),
        (1, 2): ('D', 'R', 'L', 'U'),
        (1, 3): ('D', 'U', 'L'),
        (2, 0): ('D', 'U', 'R'),
        (2, 1): ('D', 'R', 'L', 'U'),
        (2, 2): ('D', 'R', 'L', 'U'),
        (2, 3): ('D', 'U', 'L'),
        (3, 0): ('U', 'R', ),
        (3, 1): ('U', 'R', 'L'),
        (3, 2): ('U', 'R', 'L'),
        #(3, 3) ==> End-State
    }
    g.set(rewards, actions)  # setting actions and rewards in the environment
    return g


def negative_grid(step_cost):
    g = standard_grid()
    g.rewards.update({
        (0, 1): step_cost,
        (0, 2): step_cost,
        (0, 3): step_cost,
        (1, 0): step_cost,
        (1, 1): step_cost,
        (1, 2): step_cost,
        (1, 3): step_cost,
        (2, 0): step_cost,
        (2, 1): step_cost,
        (2, 2): step_cost,
        (2, 3): step_cost,
        (3, 0): step_cost,
        (3, 1): step_cost,
        (3, 2): step_cost,
    })
    return g

In [10]:
# ******************** main code body **********************

In [16]:
grid = negative_grid(-1)
print('rewards: ')
print_values(grid.rewards, grid)

pi = defaultdict(lambda: 1 / len(POSSIBLE_ACTIONS))

policy = {}
for i in grid.actions.keys():
    # assigning U/D/R/L to all the values (0,1) to (3,2)
    policy[i] = np.random.choice(POSSIBLE_ACTIONS)

Q = {}
returns = {}
# has all states from (0,0) to (3,3) -- (this list is not sorted, later we will sort it)
states = grid.all_states()

for s in states:
    if s in grid.actions:  # no terminal states included
        Q[s] = {}
        for a in POSSIBLE_ACTIONS:
            Q[s][a] = -10  # a randam value assigned
            returns[(s, a)] = []
    else:  # in case of a terminal state.
        pass


# we repeat until optimal is found
delta = []
for t in range(EPISODES):
    if (t % 1000 == 0):
        print(t)
        print("Q table: ")
        print_Q(Q, grid)

    biggest_change = 0
    states_actions_returns = play_episode(grid, policy, pi)

    # calculate Q(s,a)
    seen_state_action_pairs = set()  # initially it is empty
    # print(states_actions_returns)
    for s, a, G in states_actions_returns:
        # check if we have already seen a state s (first our var is empty so no state seen)
        # it is called "first-visit" MC policy evaluation
        sa = (s, a)
        if sa not in seen_state_action_pairs:
            old_q = Q[s][a]  # all values have -10
            returns[sa].append(G)  # we wrote return as an empty value
            # we get (state, action) --> suppose (3,2),'U':[] <-- initially []
            Q[s][a] = np.mean(returns[sa])
            m = np.abs(old_q - Q[s][a])
            biggest_change = max(biggest_change, np.abs(old_q - Q[s][a]))
            # adding to the list that has been observed
            seen_state_action_pairs.add(sa)
            # this function is returning 2 values. key and the max val of that key (key, value)
            # |
            # V
            
            A_star, _ = max_dict(Q[s])
            for a_index in POSSIBLE_ACTIONS:
                if a_index == A_star:
                    pi[(s, a_index)] = 1 - Epsilon + \
                        Epsilon / len(POSSIBLE_ACTIONS)
                else:
                    pi[(s, a_index)] = Epsilon / len(POSSIBLE_ACTIONS)
    delta.append(biggest_change)

    # calculate new policy pi(s) = argmax[a]{ Q(s,a) }
    for s in policy.keys():
        a, _ = max_dict(Q[s])
        policy[s] = a

rewards: 
---------------------------
 0.00|-1.00|-1.00|-1.00|
---------------------------
-1.00|-1.00|-1.00|-1.00|
---------------------------
-1.00|-1.00|-1.00|-1.00|
---------------------------
-1.00|-1.00|-1.00| 0.00|
0
Q table: 
----------------------------------------------
| State  | U      | D      | L      | R      |
----------------------------------------------
| (0, 0) |   0.00 |   0.00 |   0.00 |   0.00 |
| (0, 1) | -10.00 | -10.00 | -10.00 | -10.00 |
| (0, 2) | -10.00 | -10.00 | -10.00 | -10.00 |
| (0, 3) | -10.00 | -10.00 | -10.00 | -10.00 |
| (1, 0) | -10.00 | -10.00 | -10.00 | -10.00 |
| (1, 1) | -10.00 | -10.00 | -10.00 | -10.00 |
| (1, 2) | -10.00 | -10.00 | -10.00 | -10.00 |
| (1, 3) | -10.00 | -10.00 | -10.00 | -10.00 |
| (2, 0) | -10.00 | -10.00 | -10.00 | -10.00 |
| (2, 1) | -10.00 | -10.00 | -10.00 | -10.00 |
| (2, 2) | -10.00 | -10.00 | -10.00 | -10.00 |
| (2, 3) | -10.00 | -10.00 | -10.00 | -10.00 |
| (3, 0) | -10.00 | -10.00 | -10.00 | -10.00 |
| (3, 1) | -10